# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
def draw_lines0(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

""" #-------------------------------------------------------------------
P161206_2354_HoughTransfExtra.py
Created on Tue Dec  6 23:55:07 2016
@author: mo
""" #-------------------------------------------------------------------

def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    """
    GOAL:
        * extrapoler les lignes 
    SPEUDO CODE:
        * [tbc] store middle point of each line
        * identify lines: laneRight and laneRight
        * sort lineR and lineL
        * plot splineR and splineL
    TACTIC:
        * 1st, make a POC "draw a spline from a set of point"
    DOCU:   
        * np.polyfit(p1,p2) : A.X + B
    """
    laneRight = [];
    laneLeft = [];
    yRmax = 0 ;
    yLmax = 0 ;
    listR0 = [];
    listL0 = [];
    for line in lines:
        for x1,y1,x2,y2 in line:
            # identify lines: lineR and lineL
            #   compute delta
            try:
                delta = (y2-y1)/(x2-x1);
            except ZeroDivisionError:
                print('Can''t divide by zero');
                delta = 0;
            finally:
            #   if delta > 0, then add line to laneLeft
                if delta > 0:
                    laneLeft.append([(x1,y1),(x2,y2)]);
                    if yLmax < y1:
                        yLmax = y1;
                        listL0 = [x1,y1,x2,y2,'L',delta,1];
                    elif yLmax < y2:
                        yLmax = y2;
                        listL0 = [x1,y1,x2,y2,'L',delta,2];
            #   if delta < 0, then add line to laneRight
                elif delta < 0:
                    laneRight.append([(x1,y1),(x2,y2)]);
                    if yRmax < y1:
                        yRmax = y1;
                        listR0 = [x1,y1,x2,y2,'R',delta,1];
                    elif yRmax < y2:
                        yRmax = y2;
                        listR0 = [x1,y1,x2,y2,'R',delta,2];

    # sort laneRight and laneLeft
    laneLeft.sort();
    laneRight.reverse();

    # fill gap by adding new lines to the laneLeft
    laneLeft1 = [];
    i = 0;
    for line in laneLeft:
        laneLeft1.append(line);
        #insert the missing line
        i += 1;
        if i < len(laneLeft):
            laneLeft1.append([line[1],laneLeft[i][0]]);
    # fill gap by adding new lines to the laneRight
    laneRight1 = [];
    i = 0;
    for line in laneRight:
        laneRight1.append(line);
        #insert the missing line
        i += 1;
        if i < len(laneRight):
            laneRight1.append([line[1],laneRight[i][0]]); 
    
    # add the missing line from the laneX and the bottom of the image
    #   retrieve LyMax and RyMax 
    if listL0[6] == 1:
        LyMax = listL0[1];
        LxMax = listL0[0];
        print('LyMax =', LyMax); 
        print('listL0[6] =',listL0[6])
    elif listL0[6] == 2:
        LyMax = listL0[3];
        LxMax = listL0[2];
    if listR0[6] == 1:
        RyMax = listR0[1];
        RxMax = listR0[0];
    elif listR0[6] == 2:
        RyMax = listR0[3];
        RxMax = listR0[2];
    #   cas 1: LyMax > RyMax (Left lane)
    if LyMax > RyMax:
        B  = listR0[1] - listR0[5]*listR0[0];
        pRx = int((LyMax - B)/listR0[5]);
        pRy = LyMax;
        line2add = [(RxMax,RyMax),(pRx,pRy)];
        laneRight1.append(line2add);
    #   cas 2: LyMax < RyMax (Right lane)
    elif LyMax < RyMax: 
        B  = listL0[1] - listL0[5]*listL0[0];
        pLx = int((RyMax - B)/listL0[5]);
        pLy = RyMax;
        line2add = [(LxMax,LyMax),(pLx,pLy)];
        laneLeft1.append(line2add);
        
    # plot splineR and splineL
    #   fill gap by adding new lines
    for line in laneLeft1:
        cv2.line(img, line[0], line[1], color, thickness);
    for line in laneRight1:
        cv2.line(img, line[0], line[1], color, thickness);   

In [ ]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [ ]:
# ---------------------------------------------------------------------------    
# STEP 1.1 - Finding Lane Lines / IMAGE
# ---------------------------------------------------------------------------  
def FindingLaneLinesImg(image):
    #Convert to grayscale
    gray = grayscale(image);

    #Suppress noise and spurious gradients (Gaussian smoothing)
    # Define a kernel size and apply Gaussian smoothing
    blur_gray = gaussian_blur(gray,5);
    
    #Identify the edges with the Canny fct
    edges = canny(blur_gray, 50, 150);

    #Define a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320),\
                           (imshape[1],imshape[0])]], dtype=np.int32);

    #Create a masked edges image using cv2.fillPoly()
    masked_edges = region_of_interest(edges, vertices);
       
    #Hough transform
    hough_lines0 = hough_lines(masked_edges, 2, np.pi/180, 15, 40, 20)

    #Draw the lines on the edge image
    weighted_img0 = weighted_img(hough_lines0, image, α=0.8, β=1., λ=0.)

    #return gray;
    #return blur_gray;
    #return edges;
    #return masked_edges;
    #return hough_lines0;
    return weighted_img0;

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
# ---------------------------------------------------------------------------    
# STEP 1.2 - TEST : Finding Lane Lines / IMAGE
# ---------------------------------------------------------------------------  
import os
imgName = os.listdir('/home/linus/CarND-LaneLines-P1/test_images/')

for i0 in imgName:
    image = mpimg.imread('/home/linus/CarND-LaneLines-P1/test_images/'+str(i0));
    plt.figure(i0);
    plt.imshow(FindingLaneLinesImg(image));


run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    #Import packages
    result = FindingLaneLinesImg(image)
       
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = "white.mp4"
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


# 161209-1158:

I don't see myself sitting over there. I have the feeling that I have too much gaps in my skills and knowloedges for this course. It took me too much hours to get a poor result at the end. It is not a problem because I am learning a lot and a fast pace from where I started.

When I review the Param Aggarwal's feedbacks about what we learnt, I am impressed about the hindsight he has developed. I am not yet at that point. I hope I will be soon there.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))